https://platform.openai.com/docs/guides/function-calling

In [16]:
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI
import pandas as pd
import json

In [2]:
load_dotenv()

API_KEY = os.getenv('API_KEY')

In [3]:
client = OpenAI(
    api_key=os.environ.get("API_KEY"),
)

In [11]:
response = client.chat.completions.create(
    model = 'gpt-4-turbo', 
    messages = [{"role": "system", "content": 'You are a helpful assistant which informs about temperature'}, # system: tell the agent what role it needs to play
                {"role": "user", "content": 'Hey there'}],  
    temperature = 0.1
    )
print(response.choices[0].message.content)

Hello! How can I assist you today?


#### https://rapidapi.com/MeteosourceWeather/api/ai-weather-by-meteosource

In [19]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      "X-RapidAPI-Key": "b135cb7c3emsha69d64dddb4aae4p1c74b6jsn969c589c8bda",
      "X-RapidAPI-Host": "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    response = json.dumps(response.json(), indent=4)
    
    print(response)
    return response

In [20]:
response=get_current_weather('Beijing')

[
    {
        "name": "Beijing",
        "place_id": "beijing",
        "adm_area1": "Beijing Municipality",
        "adm_area2": null,
        "country": "China",
        "lat": "39.9075N",
        "lon": "116.39723E",
        "timezone": "Asia/Shanghai",
        "type": "settlement"
    },
    {
        "name": "Kitazakai",
        "place_id": "kitazakai-8311269",
        "adm_area1": "Miyagi",
        "adm_area2": "Ishinomaki Shi",
        "country": "Japan",
        "lat": "38.47681N",
        "lon": "141.30933E",
        "timezone": "Asia/Tokyo",
        "type": "settlement"
    },
    {
        "name": "Beijing",
        "place_id": "beijing-8404324",
        "adm_area1": "Chongqing Municipality",
        "adm_area2": null,
        "country": "China",
        "lat": "30.72608N",
        "lon": "108.67483E",
        "timezone": "Asia/Shanghai",
        "type": "settlement"
    },
    {
        "name": "Beijing",
        "place_id": "beijing-8212997",
        "adm_area1": "Guangd

In [22]:
# This is for the OPENAPI to understand
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'}},
   'required': ['location']}}]

In [23]:
user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})
completion=client.chat.completions.create(
    model = "gpt-4-turbo",
    messages = messages
    
)

In [25]:
messages

[{'role': 'user', 'content': 'Hi There'}]

In [24]:
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


In [26]:
user_message="What is the temperature of Boston"

messages.append({"role": "user", "content": user_message})
completion=client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    functions=functions
    
)

In [49]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Boston'}]

In [50]:
response=completion.choices[0].message
response

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None)

In [48]:
import json
location=eval(response.function_call.arguments)['location'] # eval: convert string to JSON
print(location)

Boston, MA


In [59]:
function_name=response.function_call.name
function_name

'get_current_weather'

## Send the info on the function call and function response to GPT

In [60]:
messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Boston'},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None),
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Boston, MA'}]

In [64]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Boston'},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None),
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Boston, MA'}]

In [62]:
# extend conversation with function response
second_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response

In [63]:

print(second_response.choices[0].message)

ChatCompletionMessage(content='The current temperature in Boston, MA is not specified in the response. Is there anything else you need help with?', role='assistant', function_call=None, tool_calls=None)
